In [94]:
from tensorflow.examples.tutorials.mnist import input_data

In [95]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [96]:
import tensorflow as tf

In [97]:
x = tf.placeholder("float", shape=[None, 784]) # input image vector

In [98]:
y_ = tf.placeholder("float", shape=[None, 10]) # result 0  - 9

In [99]:
x_image = tf.reshape(x, [-1,28,28,1]) # make image 28*28 matrix

In [100]:
def weight_variable(shape): # kernel weight
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

In [101]:
def bias_variable(shape): # kernel bias
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [102]:
def conv2d(x, w): # convolution, strides one by one, same size
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

In [103]:
def max_pool_2x2(x): # compression 2x2 -> 1x1
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    

In [104]:
w_conv1 = weight_variable([5, 5, 1, 32]) # first convolution kernel weight, * 32

In [105]:
b_conv1 = bias_variable([32]) # first convolution kernel bias

In [106]:
h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1) # make hidden layer

In [107]:
h_pool1 = max_pool_2x2(h_conv1) # first polling 

In [108]:
w_conv2 = weight_variable([5, 5, 32, 64]) # second convolution kernel weight, *64

In [109]:
b_conv2 = bias_variable([64]) # second convolution kernel bias

In [110]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2) # make hidden layer second time

In [111]:
h_pool2 = max_pool_2x2(h_conv2) # second pooling

In [112]:
w_fc1 = weight_variable([7 * 7 * 64, 1024]) # final weight

In [113]:
b_fc1 = bias_variable([1024]) # final bias

In [114]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) # vectorize second pooling data

In [115]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

In [116]:
keep_prob = tf.placeholder("float")

In [117]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) #  prevent overfitting

In [118]:
w_fc2 = weight_variable([1024, 10])

In [119]:
b_fc2 = bias_variable([10])

In [120]:
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2) # use softmax

In [121]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))

In [122]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [123]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))

In [124]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [125]:
sess = tf.Session()

In [126]:
sess.run(tf.global_variables_initializer())

In [135]:
for i in  range(200):
    batch = mnist.train.next_batch(50)
    if i%10 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step, feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})    

step 0, training accuracy 0.96
step 10, training accuracy 0.98
step 20, training accuracy 0.92
step 30, training accuracy 0.98
step 40, training accuracy 0.94
step 50, training accuracy 0.92
step 60, training accuracy 0.96
step 70, training accuracy 0.96
step 80, training accuracy 0.92
step 90, training accuracy 0.9
step 100, training accuracy 0.98
step 110, training accuracy 0.96
step 120, training accuracy 0.98
step 130, training accuracy 0.96
step 140, training accuracy 0.96
step 150, training accuracy 0.98
step 160, training accuracy 0.98
step 170, training accuracy 0.96
step 180, training accuracy 0.94
step 190, training accuracy 0.94


In [136]:
print("test accuracy %g"% sess.run(accuracy, feed_dict={
       x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9618
